In [110]:
// do anything to start a livy session
print("Ready? Go!")

println(util.Properties.versionMsg)

Ready? Go!Scala library version 2.11.12 -- Copyright 2002-2017, LAMP/EPFL


In [111]:
%%configure -f
{
    "conf": {
        "spark.hadoop.hadoop.security.credential.provider.path": "jceks://hdfs/user/root/awsqakeyfile.jceks",
        "spark.driver.maxResultSize": "1g",
        "spark.driver.memory": "16g",
        "spark.executor.cores": "5",
        "spark.executor.memory": "16g",
        "spark.dynamicAllocation.maxExecutors": "16"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
654,application_1581906474475_108484,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
654,application_1581906474475_108484,spark,idle,Link,Link,✔


In [112]:
import com.fasterxml.jackson.databind.{JsonNode, ObjectMapper}
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper
import org.apache.spark.sql.{DataFrame, SparkSession}

val spark = SparkSession.builder().appName("SparkSession").getOrCreate()
val checkpointDir = ""

// rememer cleanup checkpoint dir regularly
if (checkpointDir.length > 0) {
  spark.sparkContext.setCheckpointDir(checkpointDir)
}

import com.fasterxml.jackson.databind.{JsonNode, ObjectMapper}
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper
import org.apache.spark.sql.{DataFrame, SparkSession}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3937e276
checkpointDir: String = ""


### Read

In [113]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SaveMode

/*
 * configure tenant & ids
 */
val TENANT = "slin_ga_test_04"
val SYS_ACC_ID = "user_DefaultSystem_5nzu3m1p_AccountId"
val SYS_CONTACT_ID = "user_DefaultSystem_j1wuba21_ContactId"

/*
 * configure hdfs/s3 root path
 */
val HDFS_ROOT = s"hdfs:///Pods/QA/Contracts/$TENANT/Tenants/$TENANT/Spaces/Production/Data/Tables/"
val S3_ROOT = s"s3a://latticeengines-qa-customers/$TENANT/atlas/Data/Tables/"

val copySuffixFn = (df: DataFrame, cols: Seq[String], srcSuffix: String, tgtSuffix: String) => {
    var changed = false
    var res = df
    cols.filter(_.endsWith(srcSuffix)) foreach { id =>
        val dst = id.replace(srcSuffix, tgtSuffix)
        if (cols.contains(dst)) {
            println(s"=== skip copying [$id] to [$dst], target attr already exist in table")
        } else {
            changed = true
            res = res.withColumn(dst, res.col(id))
            println(s"=== copy attribute [$id] to [$dst]")
        }
    }
    (res, changed)
}

import org.apache.spark.sql.functions._
import org.apache.spark.sql.SaveMode
TENANT: String = slin_ga_test_04
SYS_ACC_ID: String = user_DefaultSystem_5nzu3m1p_AccountId
SYS_CONTACT_ID: String = user_DefaultSystem_j1wuba21_ContactId
HDFS_ROOT: String = hdfs:///Pods/QA/Contracts/slin_ga_test_04/Tenants/slin_ga_test_04/Spaces/Production/Data/Tables/
S3_ROOT: String = s3a://latticeengines-qa-customers/slin_ga_test_04/atlas/Data/Tables/
copySuffixFn: (org.apache.spark.sql.DataFrame, Seq[String], String, String) => (org.apache.spark.sql.DataFrame, Boolean) = <function4>


In [81]:
val s3Path = S3_ROOT + "Account_2020-03-09_07-27-59_UTC/*.avro"
val accounts = spark.read.format("avro").load(s3Path)

// val dedupAccs = accounts.dropDuplicates("EntityId")
// dedupAccs.write.format("avro").mode("Overwrite").save(S3_ROOT + "Account_2020-03-09_07-27-59_UTC")
println("\n===test=== " + accounts.count)

s3Path: String = s3a://latticeengines-qa-customers/slin_ga_test_03/atlas/Data/Tables/Account_2020-03-09_07-27-59_UTC/*.avro
accounts: org.apache.spark.sql.DataFrame = [EntityId: string, CustomerAccountId: string ... 29 more fields]

===test=== 900


In [114]:
/*
 * account batch store (configure table name)
 */
val accBatchTable = "Account_2020-03-10_06-55-23_UTC"
val s3Path = S3_ROOT + accBatchTable
val accounts = spark.read.format("avro").load(s3Path + "/*.avro")
println("\n=== Total Records Before: " + accounts.count)
println("=== Cols Before: " + accounts.columns.toSeq.size)
val cols = accounts.columns.toSeq
if (!cols.contains(SYS_ACC_ID)) {
    println(s"no system account id $SYS_ACC_ID found in path $s3Path")
    val accountsWithId = accounts.withColumn(SYS_ACC_ID, accounts.col("CustomerAccountId"))
    println("cols after change: " + accountsWithId.columns.toSeq)
    println(accountsWithId.take(1).head)
    println("\n=== Total Records After: " + accounts.count)
    println("=== Cols After: " + accountsWithId.columns.toSeq.size + ", " + accountsWithId.columns.toSeq)
    accountsWithId.write.format("avro").mode("Overwrite").save(s3Path + "-fix")
} else {
    println(s"=== no change")
}

accBatchTable: String = Account_2020-03-10_06-55-23_UTC
s3Path: String = s3a://latticeengines-qa-customers/slin_ga_test_04/atlas/Data/Tables/Account_2020-03-10_06-55-23_UTC
accounts: org.apache.spark.sql.DataFrame = [EntityId: string, CustomerAccountId: string ... 29 more fields]

=== Total Records Before: 900
=== Cols Before: 31
cols: Seq[String] = WrappedArray(EntityId, CustomerAccountId, Industry, SpendAnalyticsSegment, Website, CompanyName, City, State, Country, PostalCode, user_fix_du_tree_flag, user_street, user_sales_in__b, user_total_employees, user_ceo_1st_name, user_ceo_last_name, user_ceo_title, user_salesforceaccountid, user_salesforcesandboxaccountid, user_marketoaccountid, user_strange_attr, user_overlap_attr, user_test_date, user_test_date_2, user_test_date_3, user_test_date_4, InternalId, AccountId, LatticeAccountId, CDLCreatedTime, CDLUpdatedTime)
no system account id user_DefaultSystem_5nzu3m1p_AccountId found in path s3a://latticeengines-qa-customers/slin_ga_test_04/

In [115]:
/*
 * account system batch store (configure table name)
 */
val accSysTable = "SystemAccount_2020-03-10_06-55-23_UTC"
val accSysPath = S3_ROOT + accSysTable
val accSys = spark.read.format("avro").load(accSysPath + "/*.avro")
val cols = accSys.columns.toSeq

println("\n=== Total Records Before: " + accSys.count)
println("=== Cols Before: " + accSys.columns.toSeq.size)

val result = copySuffixFn(accSys, cols, "CustomerAccountId", SYS_ACC_ID)

val suffixes = Seq("CustomerAccountId", SYS_ACC_ID)

if (result._2) {
    println("\n=== cols after change: " + result._1.columns.toSeq
            .filter(col => suffixes.foldLeft(false)((acc, s) => acc || col.endsWith(s))))
    println("=== Total Records After: " + result._1.count)
    println("=== Cols After: " + result._1.columns.toSeq.size + ", " + result._1.columns.toSeq)
    result._1.write.format("avro").mode("Overwrite").save(accSysPath + "-fix")
} else {
    println("=== no change")
}

accSysTable: String = SystemAccount_2020-03-10_06-55-23_UTC
accSysPath: String = s3a://latticeengines-qa-customers/slin_ga_test_04/atlas/Data/Tables/SystemAccount_2020-03-10_06-55-23_UTC
accSys: org.apache.spark.sql.DataFrame = [SourceFile_file_1583820472295_csv__CustomerAccountId: string, SourceFile_file_1583820472295_csv__Industry: string ... 30 more fields]
cols: Seq[String] = WrappedArray(SourceFile_file_1583820472295_csv__CustomerAccountId, SourceFile_file_1583820472295_csv__Industry, SourceFile_file_1583820472295_csv__SpendAnalyticsSegment, SourceFile_file_1583820472295_csv__Website, SourceFile_file_1583820472295_csv__CompanyName, SourceFile_file_1583820472295_csv__City, SourceFile_file_1583820472295_csv__State, SourceFile_file_1583820472295_csv__Country, SourceFile_file_1583820472295_csv__PostalCode, SourceFile_file_1583820472295_csv__user_fix_du_tree_flag, SourceFile_file_1583820472295_csv__user_street, SourceFile_file_1583820472295_csv__user_sales_in__b, SourceFile_file_158382

### Compute

### Write

In [118]:
/*
 * contact batch store (configure table name)
 */
val contactBatchStore = "Contact_2020-03-10_08-26-38_UTC"
val s3Path = S3_ROOT + contactBatchStore
val contacts = spark.read.format("avro").load(s3Path + "/*.avro")
val cols = contacts.columns.toSeq

println("\n=== Total Records Before: " + contacts.count)
println("=== Cols Before: " + contacts.columns.toSeq.size + ", " + contacts.columns.toSeq)

if (!cols.contains(SYS_ACC_ID) && !cols.contains(SYS_CONTACT_ID)) {
    println(s"no system account id [$SYS_ACC_ID] and system contact id [$SYS_CONTACT_ID] found in path $s3Path")
    val contactsWithId = contacts
        .withColumn(SYS_ACC_ID, contacts.col("CustomerAccountId"))
        .withColumn(SYS_CONTACT_ID, contacts.col("CustomerContactId"))
    println("=== cols after change: " + contactsWithId.columns.toSeq.filter(col => col.equals(SYS_ACC_ID) || col.endsWith(SYS_CONTACT_ID)))
    println("\n=== Total Records After: " + contacts.count)
    println("=== Cols After: " + contactsWithId.columns.toSeq.size + ", " + contactsWithId.columns.toSeq)
    contactsWithId.write.format("avro").mode("Overwrite").save(s3Path + "-fix")
} else {
    println("=== no change")
}

contactBatchStore: String = Contact_2020-03-10_08-26-38_UTC
s3Path: String = s3a://latticeengines-qa-customers/slin_ga_test_04/atlas/Data/Tables/Contact_2020-03-10_08-26-38_UTC
contacts: org.apache.spark.sql.DataFrame = [EntityId: string, CustomerContactId: string ... 25 more fields]
cols: Seq[String] = WrappedArray(EntityId, CustomerContactId, ContactName, FirstName, LastName, CustomerAccountId, Title, NumberOfEmployees, Industry, DoNotMail, CreatedDate, Email, PostalCode, Address_Street_1, user_gender, user_ip_address, user_some_description, user_last_communication_date, user_renewal_date, InternalId, ContactId, AccountId, LatticeAccountId, CDLCreatedTime, CDLUpdatedTime, user_DefaultSystem_5nzu3m1p_AccountId, user_DefaultSystem_j1wuba21_ContactId)

=== Total Records Before: 900
=== Cols Before: 27, WrappedArray(EntityId, CustomerContactId, ContactName, FirstName, LastName, CustomerAccountId, Title, NumberOfEmployees, Industry, DoNotMail, CreatedDate, Email, PostalCode, Address_Stree

In [117]:
/*
 * contact system batch store (configure table name)
 */
val contactSysTable = "SystemContact_2020-03-10_08-26-38_UTC"
val s3Path = S3_ROOT + contactSysTable
val contacts = spark.read.format("avro").load(s3Path + "/*.avro")
val cols = contacts.columns.toSeq

println("=== " + cols.filter(col => col.endsWith("CustomerContactId")))
println("\n=== Total Records Before: " + contacts.count)
println("=== Cols Before: " + contacts.columns.toSeq.size)

val resultAcc = copySuffixFn(contacts, cols, "CustomerAccountId", SYS_ACC_ID)
val result = copySuffixFn(resultAcc._1, cols, "CustomerContactId", SYS_CONTACT_ID)

val suffixes = Seq("CustomerAccountId", SYS_ACC_ID, "CustomerContactId", SYS_CONTACT_ID)

if (resultAcc._2 || result._2) {
    println("=== cols after change: " + result._1.columns.toSeq
            .filter(col => suffixes.foldLeft(false)((acc, s) => acc || col.endsWith(s))))
    println("\n=== Total Records after: " + result._1.count)
    println("=== Cols after: " + result._1.columns.toSeq.size + ", " + result._1.columns.toSeq)
    result._1.write.format("avro").mode("Overwrite").save(s3Path + "-fix")
} else {
    println("=== no change")
}

contactSysTable: String = SystemContact_2020-03-10_08-26-38_UTC
s3Path: String = s3a://latticeengines-qa-customers/slin_ga_test_04/atlas/Data/Tables/SystemContact_2020-03-10_08-26-38_UTC
contacts: org.apache.spark.sql.DataFrame = [SourceFile_file_1583820509665_csv__CustomerContactId: string, SourceFile_file_1583820509665_csv__ContactName: string ... 24 more fields]
cols: Seq[String] = WrappedArray(SourceFile_file_1583820509665_csv__CustomerContactId, SourceFile_file_1583820509665_csv__ContactName, SourceFile_file_1583820509665_csv__FirstName, SourceFile_file_1583820509665_csv__LastName, SourceFile_file_1583820509665_csv__CustomerAccountId, SourceFile_file_1583820509665_csv__Title, SourceFile_file_1583820509665_csv__NumberOfEmployees, SourceFile_file_1583820509665_csv__Industry, SourceFile_file_1583820509665_csv__DoNotMail, SourceFile_file_1583820509665_csv__CreatedDate, SourceFile_file_1583820509665_csv__Email, SourceFile_file_1583820509665_csv__PostalCode, SourceFile_file_158382050966